In [1]:
import uproot
import glob
import json
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import numpy as np
import itertools

from utils.analysis import PandasAnalysis
from utils.cutflow import Cutflow

TAG = "pku"

In [2]:
babies = glob.glob(f"../analysis/studies/vbswh/output_{TAG}/Run2/*.root")
sig_babies = [b for b in babies if "VBSWH_mkW" in b]
bkg_babies = [b for b in babies if "VBSWH_mkW" not in b and "VBFWH" not in b and "data" not in b]
data_babies = [b for b in babies if "data.root" in b]
babies

['../analysis/studies/vbswh/output_pku/Run2/WJets.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTX.root',
 '../analysis/studies/vbswh/output_pku/Run2/VH.root',
 '../analysis/studies/vbswh/output_pku/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_pku/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_pku/Run2/SingleTop.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTbar1L.root',
 '../analysis/studies/vbswh/output_pku/Run2/VBSWH_SM.root',
 '../analysis/studies/vbswh/output_pku/Run2/VBSWH_mkW.root',
 '../analysis/studies/vbswh/output_pku/Run2/data.root']

In [3]:
vbswh = PandasAnalysis(
    sig_root_files=sig_babies,
    bkg_root_files=bkg_babies,
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=[
        "xsec_sf", "lep_id_sf", "elec_reco_sf", "muon_iso_sf", 
        "btag_sf", "pu_sf", "prefire_sf", "trig_sf"
    ]
)
vbswh.df["unity"] = 1 # IMPORTANT
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)
vbswh.df["SR2"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR1"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR2_up"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_up > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR1_up"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_up > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR2_dn"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_dn > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR1_dn"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_dn > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["regionA"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
)
vbswh.df["regionB"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
)
vbswh.df["regionC"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
)

SIGNAL_REGIONS = ["SR1", "SR2"]

Loading data babies: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.10s/it]


# Utilities

In [4]:
class Systematic:
    def __init__(self, name, signal_regions):
        self.name = name
        self.signal_regions = signal_regions
        self.systs = {signal_region: [] for signal_region in signal_regions}
        
    def copy(self, name):
        new_systs = Systematic(name, signal_regions=self.signal_regions)
        new_systs.systs = self.systs
        return new_systs

    def add_syst(self, syst, signal_region):
        self.systs[signal_region].append(syst)
        
    def add_systs(self, systs, signal_region=None):
        if signal_region:
            self.systs[signal_region] += systs
        else:
            for syst_i, syst in enumerate(systs):
                self.systs[self.signal_regions[syst_i]].append(syst)
    
    def get_systs(self):
        return self.systs
    
    def get_systs_str(self, signal_region=None):
        if signal_region:
            systs = self.systs[signal_region]
            if len(systs) == 1:
                return f"{systs[0]:0.1%}"
            else:
                return f"{min(systs):0.1%} - {max(systs):0.1%}"
        else:
            return {SR: self.get_systs_str(signal_region=SR) for SR in self.signal_regions}
        
class SystematicsTable:
    def __init__(self, systs=None, samples=None):
        self.systs = systs or []
        self.samples = samples or []
        
    def add_row(self, syst):
        self.systs.append(syst)
        
    def to_dataframe(self):
        rows = []
        for syst in self.systs:
            row = {"Systematic": syst.name}
            row.update(syst.get_systs_str())
            rows.append(row)
        return pd.DataFrame(rows)
        
    def to_latex(self, output_tex=None):
        # Convert to Pandas DataFrame
        df = self.to_dataframe()
        
        # Convert to LaTeX
        latex = (df.style
                   .hide(axis="index")
                   .to_latex(column_format="lcc", position="H")
                   .replace("%", "\%"))
        # Insert hlines and centering
        latex = latex.split("\n")
        latex.insert(3, "\\hline")
        latex.insert(2, "\\hline\n\\hline")
        latex.insert(1, "\\begin{center}")
        latex.insert(-3, "\\hline\n\\hline")
        latex.insert(-2, "\\end{center}")
        latex = "\n".join(latex)
        
        return latex
    
    def to_datacard_json(self, signal_regions=None, output_json=None):
        datacard_systs = {}
        for syst in self.systs:
            labeled_systs = syst.get_systs()
            datacard_systs[syst.name] = [1 + max(systs[SR]) for SR in signal_regions]
            
        if output_json:
            with open(output_json, "w") as json_out:
                json.dump(datacard_systs, json_out)
                
        return datacard_systs

# Scale factors

In [5]:
SIG_SYSTS_TABLE = SystematicsTable(samples=["VBSWH_mkW"])
BKG_SYSTS_TABLE = SystematicsTable(
    samples=[b.split("/")[-1].replace(".root", "") for b in bkg_babies]
)
SIG_SYSTS_LIMIT = SystematicsTable(samples=["VBSWH_mkW"])

In [6]:
def get_systs(sample_name, signal_regions, sf, *sf_variations):
    global vsbwh
    systs = []
    for SR in signal_regions:
        df = vbswh.sample_df(name=sample_name, selection=SR)
        # Get nominal
        count = df.event_weight.sum()
        # Get delta = nominal - variation for each variation
        deltas = []
        for sf_var in sf_variations:
            count_var = np.sum(df.event_weight/df[sf]*df[sf_var])
            deltas.append(abs((count - count_var)/count))
        
        systs.append(max(deltas))
        
    return systs

def get_systs_nonSF(sample_name, signal_regions, signal_regions_up, signal_regions_dn):
    systs = []
    for SR_i, SR in enumerate(signal_regions):
        SR_up = signal_regions_up[SR_i]
        SR_dn = signal_regions_dn[SR_i]
        
        df = vbswh.sample_df(name=sample_name, selection=SR)
        df_up = vbswh.sample_df(name=sample_name, selection=SR_up)
        df_dn = vbswh.sample_df(name=sample_name, selection=SR_dn)
        
        count = df.event_weight.sum()
        count_up = df_up.event_weight.sum()
        count_dn = df_dn.event_weight.sum()
        
        up_perc = abs((count_up - count)/count)
        dn_perc = abs((count - count_dn)/count)
        
        systs.append(max(up_perc, dn_perc))
        
    return systs

### PDF uncertainty

In [7]:
root_files = glob.glob("/ceph/cms/store/user/jguiang/VBSVHSkim/sig_1lep_1ak8_2ak4_pku/VBSWH*NANOGEN*/merged.root")
for root_file in root_files:
    gen_sum = 0
    pdf_sum = np.zeros(103)
    with uproot.open(root_file) as f:
        gen_sums = f["Runs"]["genEventSumw"].array(library="np")
        pdf_sums = f["Runs"]["LHEPdfSumw"].array(library="np")
        missed = np.array([len(s) != 103 for s in pdf_sums])
        reshaped = np.vstack(pdf_sums[~missed])
        pdf_sum += np.dot(gen_sums[~missed], reshaped) + np.sum(gen_sums[missed])
        gen_sum += np.sum(gen_sums)
        
pdf_ratio = pdf_sum/gen_sum

with uproot.open("../analysis/studies/vbswh/output_pku/Run2/VBSWH_mkW.root") as f:
    df = f.get("pdf_tree").arrays(library="pd")
    
systs = []
for signal_region in SIGNAL_REGIONS:
    SR = df.eval(signal_region)
    count = df[SR].event_weight.sum()
    deltas = []
    for i in range(1, 101):
        count_var = np.sum(df[SR].event_weight*df[SR][f"lhe_pdf_{i}"])
        deltas.append(count - count_var/pdf_ratio[i])

    deltas = np.array(deltas)
    systs.append(np.sqrt(np.sum(deltas**2))/count)

pdf_systs = Systematic("PDF variations", SIGNAL_REGIONS)
pdf_systs.add_systs(systs)
SIG_SYSTS_TABLE.add_row(pdf_systs)
SIG_SYSTS_LIMIT.add_row(pdf_systs.copy("pdf_vars"))

### $\alpha_S$ uncertainty

In [8]:
vbswh.df["alphaS_corr_up"] = vbswh.df.alphaS_up
vbswh.df["alphaS_corr_dn"] = vbswh.df.alphaS_dn

vbswh.df.loc[vbswh.df.is_signal, "alphaS_corr_up"] = vbswh.df[vbswh.df.is_signal].alphaS_up/pdf_ratio[101]
vbswh.df.loc[vbswh.df.is_signal, "alphaS_corr_dn"] = vbswh.df[vbswh.df.is_signal].alphaS_up/pdf_ratio[102]

alphaS_systs = Systematic("$\\alpha_S$ unc.", SIGNAL_REGIONS)
alphaS_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", "alphaS_corr_dn", "alphaS_corr_up")
)
SIG_SYSTS_TABLE.add_row(alphaS_systs)
SIG_SYSTS_LIMIT.add_row(alphaS_systs.copy("alphaS_unc"))

### LHE scale weights

In [9]:
lhe_muR_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("muF1p0")])
lhe_muF_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("muR1p0")])

muR_systs = Systematic("$\\mu_R$ scale", SIGNAL_REGIONS)
muR_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *lhe_muR_weights)
)

muF_systs = Systematic("$\\mu_F$ scale", SIGNAL_REGIONS)
muF_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *lhe_muF_weights)
)

# SIG_SYSTS_TABLE.add_row(muR_systs)                   # muR variations have not effect
SIG_SYSTS_TABLE.add_row(muF_systs)
# SIG_SYSTS_LIMIT.add_row(muR_systs.copy("muR_scale")) # muR variations have not effect
SIG_SYSTS_LIMIT.add_row(muF_systs.copy("muF_scale"))

### Parton shower weights

In [10]:
isr_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("fsr1p0")])
fsr_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("isr1p0")])

isr_sf_systs = Systematic("Parton shower ISR weights", SIGNAL_REGIONS)
isr_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *isr_weights)
)

fsr_sf_systs = Systematic("Parton shower FSR weights", SIGNAL_REGIONS)
fsr_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *fsr_weights)
)

SIG_SYSTS_TABLE.add_row(isr_sf_systs)
SIG_SYSTS_TABLE.add_row(fsr_sf_systs)
SIG_SYSTS_LIMIT.add_row(isr_sf_systs.copy("isr_weights"))
SIG_SYSTS_LIMIT.add_row(fsr_sf_systs.copy("fsr_weights"))

### Pileup reweighting

In [11]:
bkg_pu_sf_systs = Systematic("Pileup reweighting", SIGNAL_REGIONS)

for sample_name in BKG_SYSTS_TABLE.samples:
    bkg_pu_sf_systs.add_systs(
        get_systs(sample_name, SIGNAL_REGIONS, "pu_sf", "pu_sf_dn", "pu_sf_up")
    )

BKG_SYSTS_TABLE.add_row(bkg_pu_sf_systs)

In [12]:
pu_sf_systs = Systematic("Pileup reweighting", SIGNAL_REGIONS)
pu_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "pu_sf", "pu_sf_dn", "pu_sf_up")
)
SIG_SYSTS_TABLE.add_row(pu_sf_systs)
SIG_SYSTS_LIMIT.add_row(pu_sf_systs.copy("pu_rwgt"))

### L1 Prefiring weight

In [13]:
bkg_prefire_sf_systs = Systematic("L1 pre-fire corrections", SIGNAL_REGIONS)

for sample_name in BKG_SYSTS_TABLE.samples:
    bkg_prefire_sf_systs.add_systs(
        get_systs(sample_name, SIGNAL_REGIONS, "prefire_sf", "prefire_sf_up", "prefire_sf_dn")
    )
        
BKG_SYSTS_TABLE.add_row(bkg_prefire_sf_systs)

In [14]:
prefire_sf_systs = Systematic("L1 pre-fire corrections", SIGNAL_REGIONS)
prefire_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "prefire_sf", "prefire_sf_up", "prefire_sf_dn")
)
SIG_SYSTS_TABLE.add_row(prefire_sf_systs)
SIG_SYSTS_LIMIT.add_row(prefire_sf_systs.copy("L1_prefire"))

### HLT scale factors

In [15]:
trig_sf_systs = Systematic("HLT scale factors", SIGNAL_REGIONS)
trig_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "trig_sf", "trig_sf_up", "trig_sf_dn")
)
SIG_SYSTS_TABLE.add_row(trig_sf_systs)
SIG_SYSTS_LIMIT.add_row(trig_sf_systs.copy("hlt_sfs"))

### Statistical uncertainty

In [16]:
bkg_stat_systs = Systematic("Simulation stat. unc.", SIGNAL_REGIONS)
for sample_name in BKG_SYSTS_TABLE.samples:
    bkg_stat_systs.add_systs(
        [
            vbswh.bkg_error(selection=f"SR1 and name == '{sample_name}'")/vbswh.bkg_count(selection="SR1"), 
            vbswh.bkg_error(selection=f"SR2 and name == '{sample_name}'")/vbswh.bkg_count(selection="SR2")
        ]
    )
BKG_SYSTS_TABLE.add_row(bkg_stat_systs)

In [17]:
stat_systs = Systematic("Simulation stat. unc.", SIGNAL_REGIONS)
stat_systs.add_systs(
    [
        vbswh.sig_error(selection="SR1")/vbswh.sig_count(selection="SR1"), 
        vbswh.sig_error(selection="SR2")/vbswh.sig_count(selection="SR2")
    ]
)
SIG_SYSTS_TABLE.add_row(stat_systs)
SIG_SYSTS_LIMIT.add_row(stat_systs.copy("mc_stat"))

### Lepton scale factors

In [18]:
bkg_lep_sf_systs = Systematic("Lepton scale factors", SIGNAL_REGIONS)

for sample_name in BKG_SYSTS_TABLE.samples:
    bkg_lep_sf_systs.add_systs(
        get_systs(sample_name, SIGNAL_REGIONS, "lep_id_sf", "lep_id_sf_up", "lep_id_sf_dn")
    )
        
BKG_SYSTS_TABLE.add_row(bkg_lep_sf_systs)

In [19]:
lep_sf_systs = Systematic("Lepton scale factors", SIGNAL_REGIONS)
lep_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "lep_id_sf", "lep_id_sf_up", "lep_id_sf_dn")
)
lep_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "elec_reco_sf", "elec_reco_sf_up", "elec_reco_sf_dn")
)
lep_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "muon_iso_sf", "muon_iso_sf_up", "muon_iso_sf_dn")
)
SIG_SYSTS_TABLE.add_row(lep_sf_systs)

lep_id_sf_systs = Systematic("lep_id", SIGNAL_REGIONS)
lep_id_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "lep_id_sf", "lep_id_sf_up", "lep_id_sf_dn")
)
SIG_SYSTS_LIMIT.add_row(lep_id_sf_systs)

elec_reco_sf_systs = Systematic("elec_reco", SIGNAL_REGIONS)
elec_reco_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "elec_reco_sf", "elec_reco_sf_up", "elec_reco_sf_dn")
)
SIG_SYSTS_LIMIT.add_row(elec_reco_sf_systs)

muon_iso_sf_systs = Systematic("muon_iso", SIGNAL_REGIONS)
muon_iso_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "muon_iso_sf", "muon_iso_sf_up", "muon_iso_sf_dn")
)
SIG_SYSTS_LIMIT.add_row(muon_iso_sf_systs)

### DeepJet b-tagging scale factors

In [20]:
bkg_btag_sf_systs = Systematic("DeepJet b-tagging scale factors", SIGNAL_REGIONS)

for sample_name in BKG_SYSTS_TABLE.samples:
    bkg_btag_sf_systs.add_systs(
        get_systs(sample_name, SIGNAL_REGIONS, "btag_sf", "btag_sf_dn", "btag_sf_up")
    )

BKG_SYSTS_TABLE.add_row(bkg_btag_sf_systs)

In [21]:
btag_sf_systs = Systematic("DeepJet b-tagging scale factors", SIGNAL_REGIONS)
btag_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "btag_sf", "btag_sf_dn", "btag_sf_up")
)
SIG_SYSTS_TABLE.add_row(btag_sf_systs)
SIG_SYSTS_LIMIT.add_row(btag_sf_systs.copy("btag_sfs"))

### MET uncertainty

In [22]:
bkg_met_unc_systs = Systematic("MET unc.", SIGNAL_REGIONS)

for sample_name in BKG_SYSTS_TABLE.samples:
    bkg_met_unc_systs.add_systs(
        get_systs_nonSF(sample_name, SIGNAL_REGIONS, ["SR1_dn", "SR2_dn"], ["SR1_up", "SR2_up"])
    )

BKG_SYSTS_TABLE.add_row(bkg_met_unc_systs)

In [23]:
met_unc_systs = Systematic("MET unc.", SIGNAL_REGIONS)
met_unc_systs.add_systs(
    get_systs_nonSF("VBSWH_mkW", SIGNAL_REGIONS, ["SR1_dn", "SR2_dn"], ["SR1_up", "SR2_up"])
)
SIG_SYSTS_TABLE.add_row(met_unc_systs)
SIG_SYSTS_LIMIT.add_row(met_unc_systs.copy("met_unc"))

# Jet energy

In [24]:
def get_jet_energy_systs(nominal_cflow, up_cflow, dn_cflow, signal_regions, name):
    
    nominal_cutflow = Cutflow.from_file(nominal_cflow)
    up_cutflow = Cutflow.from_file(up_cflow)
    dn_cutflow = Cutflow.from_file(dn_cflow)

    syst_up_cutflow = (up_cutflow - nominal_cutflow)/nominal_cutflow
    syst_dn_cutflow = (nominal_cutflow - dn_cutflow)/nominal_cutflow

    systs = Systematic(name, signal_regions.keys())
    for SR, cut_name in signal_regions.items():
        systs.add_syst(
            max(
                syst_up_cutflow[cut_name].n_pass,
                syst_dn_cutflow[cut_name].n_pass
            ),
            signal_region=SR
        )
        
    return systs

### Jet energy corrections

In [25]:
jec_systs = Systematic("Jet energy scale", ["SR1", "SR2"])

for sample_name in BKG_SYSTS_TABLE.samples:
    systs = get_jet_energy_systs(
        f"../analysis/studies/vbswh/output_pku/Run2/{sample_name}_cutflow.cflow",
        f"../analysis/studies/vbswh/output_pku_jec_up/Run2/{sample_name}_cutflow.cflow",
        f"../analysis/studies/vbswh/output_pku_jec_dn/Run2/{sample_name}_cutflow.cflow",
        {"SR1": "XbbGt0p9_MSDLt150", "SR2": "STGt1500"},
        "Jet energy scale"
    )
    for SR, values in systs.get_systs().items():
        jec_systs.add_systs(values, signal_region=SR)
        
BKG_SYSTS_TABLE.add_row(jec_systs)

In [26]:
jec_systs = get_jet_energy_systs(
    "../analysis/studies/vbswh/output_pku/Run2/VBSWH_mkW_cutflow.cflow",
    "../analysis/studies/vbswh/output_pku_jec_up/Run2/VBSWH_mkW_cutflow.cflow",
    "../analysis/studies/vbswh/output_pku_jec_dn/Run2/VBSWH_mkW_cutflow.cflow",
    {"SR1": "XbbGt0p9_MSDLt150", "SR2": "STGt1500"},
    "Jet energy scale"
)
SIG_SYSTS_TABLE.add_row(jec_systs)
SIG_SYSTS_LIMIT.add_row(jec_systs.copy("jes"))

### Jet energy resolution

In [27]:
jer_systs = Systematic("Jet energy resolution", ["SR1", "SR2"])

for sample_name in BKG_SYSTS_TABLE.samples:
    systs = get_jet_energy_systs(
        f"../analysis/studies/vbswh/output_pku/Run2/{sample_name}_cutflow.cflow",
        f"../analysis/studies/vbswh/output_pku_jer_up/Run2/{sample_name}_cutflow.cflow",
        f"../analysis/studies/vbswh/output_pku_jer_dn/Run2/{sample_name}_cutflow.cflow",
        {"SR1": "XbbGt0p9_MSDLt150", "SR2": "STGt1500"},
        "Jet energy resolution"
    )
    for SR, values in systs.get_systs().items():
        jer_systs.add_systs(values, signal_region=SR)
        
BKG_SYSTS_TABLE.add_row(jer_systs)

In [28]:
jer_systs = get_jet_energy_systs(
    "../analysis/studies/vbswh/output_pku/Run2/VBSWH_mkW_cutflow.cflow",
    "../analysis/studies/vbswh/output_pku_jer_up/Run2/VBSWH_mkW_cutflow.cflow",
    "../analysis/studies/vbswh/output_pku_jer_dn/Run2/VBSWH_mkW_cutflow.cflow",
    {"SR1": "XbbGt0p9_MSDLt150", "SR2": "STGt1500"},
    "Jet energy resolution"
)
SIG_SYSTS_TABLE.add_row(jer_systs)
SIG_SYSTS_LIMIT.add_row(jer_systs.copy("jer"))

# Other

### Luminosity

In [29]:
lumi_systs = Systematic("Luminosity", SIGNAL_REGIONS)
lumi_systs.add_systs([0.025, 0.025])
BKG_SYSTS_TABLE.add_row(lumi_systs)
SIG_SYSTS_TABLE.add_row(lumi_systs)
SIG_SYSTS_LIMIT.add_row(lumi_systs.copy("lumi"))

In [30]:
BKG_SYSTS_TABLE.to_dataframe()

,Systematic,SR1,SR2
0,Pileup reweighting,0.4% - 14.9%,0.4% - 8.9%
1,L1 pre-fire corrections,0.3% - 1.5%,0.1% - 1.2%
2,Simulation stat. unc.,0.0% - 2.3%,0.0% - 20.7%
3,Lepton scale factors,0.9% - 7.0%,0.9% - 3.4%
4,DeepJet b-tagging scale factors,0.0% - 3.3%,0.0% - 8.2%
5,MET unc.,0.2% - 27.3%,0.0% - 0.9%
6,Jet energy scale,9.5% - 16.1%,0.0% - 26.7%
7,Jet energy resolution,1.6% - 5.6%,0.0% - 20.0%
8,Luminosity,2.5%,2.5%


In [31]:
SIG_SYSTS_TABLE.to_dataframe()

,Systematic,SR1,SR2
0,PDF variations,17.7%,36.9%
1,$\alpha_S$ unc.,2.6%,4.4%
2,$\mu_F$ scale,17.9%,21.2%
3,Parton shower ISR weights,0.1%,0.1%
4,Parton shower FSR weights,2.6%,2.1%
5,Pileup reweighting,0.3%,0.6%
6,L1 pre-fire corrections,1.0%,1.0%
7,HLT scale factors,0.8%,0.9%
8,Simulation stat. unc.,2.1%,4.0%
9,Lepton scale factors,0.0% - 1.4%,0.0% - 1.9%


In [32]:
print(SIG_SYSTS_TABLE.to_latex())

\begin{table}[H]
\begin{center}
\begin{tabular}{lcc}
\hline
\hline
Systematic & SR1 & SR2 \\
\hline
PDF variations & 17.7\% & 36.9\% \\
$\alpha_S$ unc. & 2.6\% & 4.4\% \\
$\mu_F$ scale & 17.9\% & 21.2\% \\
Parton shower ISR weights & 0.1\% & 0.1\% \\
Parton shower FSR weights & 2.6\% & 2.1\% \\
Pileup reweighting & 0.3\% & 0.6\% \\
L1 pre-fire corrections & 1.0\% & 1.0\% \\
HLT scale factors & 0.8\% & 0.9\% \\
Simulation stat. unc. & 2.1\% & 4.0\% \\
Lepton scale factors & 0.0\% - 1.4\% & 0.0\% - 1.9\% \\
DeepJet b-tagging scale factors & 0.3\% & 0.3\% \\
MET unc. & 0.1\% & 0.8\% \\
Jet energy scale & 6.1\% & 7.5\% \\
Jet energy resolution & 0.1\% & 0.5\% \\
Luminosity & 2.5\% & 2.5\% \\
\hline
\hline
\end{tabular}
\end{center}
\end{table}



In [33]:
class Datacard:
    def __init__(self, obs, sig, bkg, systs):
        self.obs = obs
        self.n_obs = len(obs)
        
        self.sig_labels = []
        self.sig_yields = [[] for _ in range(self.n_obs)]
        for sig_label, sig_yields in sig.items():
            self.sig_labels.append(sig_label)
            for bin_i, sig_yield in enumerate(sig_yields):
                self.sig_yields[bin_i].append(sig_yield)
                
        self.bkg_labels = []
        self.bkg_yields = [[] for _ in range(self.n_obs)]
        for bkg_label, bkg_yields in bkg.items():
            self.bkg_labels.append(bkg_label)
            for bin_i, bkg_yield in enumerate(bkg_yields):
                self.bkg_yields[bin_i].append(bkg_yield)

        self.n_sig = len(self.sig_labels)
        self.n_bkg = len(self.bkg_labels)
        
        self.column_width = max([len(l) for l in self.sig_labels + self.bkg_labels])+2
        self.column_width = max(self.column_width, 12)
                
        self.syst_labels = []
        self.systs = []
        n_samples = self.n_sig + self.n_bkg
        w = self.column_width
        for sample_label, labeled_systs in systs.items():
            for syst_label, syst_values in labeled_systs.items():
                # Register syst
                if syst_label not in self.syst_labels:
                    self.syst_labels.append(syst_label)
                    self.systs.append([f"{'-':>{w}}" for _ in range(self.n_obs*(n_samples))])
                
                # Get index of syst label
                label_i = self.syst_labels.index(syst_label)
                
                # Assign syst values
                if sample_label in self.sig_labels:
                    for value_i, syst_value in enumerate(syst_values):
                        syst_i = self.sig_labels.index(sample_label)+value_i*n_samples
                        self.systs[label_i][syst_i] = f"{syst_value:{w}.3f}" 
                elif sample_label in self.bkg_labels:
                    for value_i, syst_value in enumerate(syst_values):
                        syst_i = self.n_sig+self.bkg_labels.index(sample_label)+value_i*n_samples
                        self.systs[label_i][syst_i] = f"{syst_value:{w}.3f}"  
                else:
                    raise Exception(f"{sample_label} not found")
                    
        self.header_width = max([len(l) for l in self.syst_labels])+2
        self.header_width = max(self.header_width, 12)
        
        self.content = None
        self.__create()
        
    def __create(self):
        cw = self.column_width
        hw = self.header_width
        hline = "-"*(hw+5 + cw*(self.n_sig + self.n_bkg)*self.n_obs) + "\n"
        content = ""
        content += f"imax {self.n_obs} number of channels\n"
        content += f"jmax {self.n_bkg} number of backgrounds\n"
        content += f"kmax {len(self.systs)} number of nuisance parameters\n"
        content += hline
        content += f"{'bin':<{hw+5}}"
        content +=  "".join([f"{'bin'+str(i+1):>{cw}}" for i in range(self.n_obs)])
        content +=  "\n"
        content += f"{'observation':<{hw+5}}"
        content +=  "".join([f"{n:{cw}d}" for n in self.obs])
        content +=  "\n"
        content += hline
        content += f"{'bin':<{hw+5}}"
        content += "".join([f"{'bin'+str(i+1):>{cw}}" for i in range(self.n_obs) for _ in range(self.n_sig + self.n_bkg)])
        content +=  "\n"
        content += f"{'process':<{hw+5}}"
        content +=  "".join([f"{l:>{cw}}" for _ in range(self.n_obs) for l in self.sig_labels + self.bkg_labels])
        content +=  "\n"
        content += f"{'process':<{hw+5}}"
        content += "".join([f"{i:>{cw}}" for _ in range(self.n_obs) for i in range(self.n_sig + self.n_bkg)])
        content +=  "\n"
        content += f"{'rate':<{hw+5}}"
        content +=  "".join([f"{y[i]:{cw}.2f}" for y in self.sig_yields for i in range(self.n_sig)])
        content +=  "".join([f"{y[i]:{cw}.2f}" for y in self.bkg_yields for i in range(self.n_bkg)])
        content +=  "\n"
        content += hline
        for syst_i, syst_values in enumerate(self.systs):
            content += f"{self.syst_labels[syst_i]:<{hw}}"
            content +=  " lnN "
            content +=  "".join(syst_values)
            content += "\n"
            
        self.content = content
        
    def write(self, output_dat):
        with open(output_dat, "w") as dat_out:
            dat_out.write(self.content)
        
datacard_systs = {
    "TotalBkg": {
        "abcd_syst": [1.14],
        "abcd_stat": [1.13]
    }
}

datacard_systs["VBSWH_mkW"] = {}
for syst_obj in SIG_SYSTS_LIMIT.systs:
    systs = syst_obj.get_systs()
    datacard_systs["VBSWH_mkW"][syst_obj.name] = [1 + systs["SR1"][0]]
    
pred_bkg = vbswh.data_count(selection="regionA")/vbswh.data_count(selection="regionB")*vbswh.data_count(selection="regionC")
datacard = Datacard(
    [int(pred_bkg)],
    {"VBSWH_mkW": [vbswh.sig_count(selection="SR1")]},
    {"TotalBkg": [pred_bkg]},
    datacard_systs
)
    
print(datacard.content)
datacard.write("../combine/datacards/vbswh.dat")

imax 1 number of channels
jmax 1 number of backgrounds
kmax 19 number of nuisance parameters
------------------------------------------
bin                       bin1
observation                129
------------------------------------------
bin                       bin1        bin1
process              VBSWH_mkW    TotalBkg
process                      0           1
rate                    393.53      129.50
------------------------------------------
abcd_syst     lnN            -       1.140
abcd_stat     lnN            -       1.130
pdf_vars      lnN        1.177           -
alphaS_unc    lnN        1.026           -
muF_scale     lnN        1.179           -
isr_weights   lnN        1.001           -
fsr_weights   lnN        1.026           -
pu_rwgt       lnN        1.003           -
L1_prefire    lnN        1.010           -
hlt_sfs       lnN        1.008           -
mc_stat       lnN        1.021           -
lep_id        lnN        1.014           -
elec_reco     lnN        1.0